### QUESTION 9

In [23]:
df='dataset-2.csv'

In [24]:
import pandas as pd
import numpy as np
path="dataset-2.csv"
def calculate_distance_matrix(path):
    df=pd.read_csv(path)
    a=sorted(set(df['id_start']) | set(df['id_end']))
    distance_matrix=pd.DataFrame(np.inf,index=a,columns=a)
    np.fill_diagonal(distance_matrix.values,0)
    for _, row in df.iterrows():
        start,end,distance=row['id_start'],row['id_end'],row['distance']
        distance_matrix.at[start,end]=distance_matrix.at[end,start]=distance
    for k in a:
        for i in a:
            for j in a:
                if distance_matrix.at[i, j] > distance_matrix.at[i, k] + distance_matrix.at[k, j]:
                    distance_matrix.at[i, j] = distance_matrix.at[i, k] + distance_matrix.at[k, j]
    return distance_matrix.round(1)    

In [25]:
path="dataset-2.csv"
result_matrix = calculate_distance_matrix(path)
print(result_matrix)

         1001400  1001402  1001404  1001406  1001408  1001410  1001412  \
1001400      0.0      9.7     29.9     45.9     67.6     78.7     94.3   
1001402      9.7      0.0     20.2     36.2     57.9     69.0     84.6   
1001404     29.9     20.2      0.0     16.0     37.7     48.8     64.4   
1001406     45.9     36.2     16.0      0.0     21.7     32.8     48.4   
1001408     67.6     57.9     37.7     21.7      0.0     11.1     26.7   
1001410     78.7     69.0     48.8     32.8     11.1      0.0     15.6   
1001412     94.3     84.6     64.4     48.4     26.7     15.6      0.0   
1001414    112.5    102.8     82.6     66.6     44.9     33.8     18.2   
1001416    125.7    116.0     95.8     79.8     58.1     47.0     31.4   
1001418    139.3    129.6    109.4     93.4     71.7     60.6     45.0   
1001420    152.2    142.5    122.3    106.3     84.6     73.5     57.9   
1001422    161.8    152.1    131.9    115.9     94.2     83.1     67.5   
1001424    173.2    163.5    143.3    

### QUESTION 10

In [26]:
def unroll_distance_matrix(distance_matrix):
    b=distance_matrix.index.tolist()
    x,y,z=[],[],[]
    for id_start in b:
        for id_end in b:
            if id_start!=id_end:
                x.append(id_start)
                y.append(id_end)
                z.append(distance_matrix.at[id_start,id_end])
                
    result_df=pd.DataFrame({'id_start':x,'id_end':y,'distance':z})
    return result_df

In [31]:
distance_matrix=calculate_distance_matrix(df)
unroll_df=unroll_distance_matrix(distance_matrix)
print(unroll_df)

      id_start   id_end  distance
0      1001400  1001402       9.7
1      1001400  1001404      29.9
2      1001400  1001406      45.9
3      1001400  1001408      67.6
4      1001400  1001410      78.7
...        ...      ...       ...
1801   1004356  1001470     159.8
1802   1004356  1001472     175.8
1803   1004356  1001488       4.0
1804   1004356  1004354       2.0
1805   1004356  1004355       4.0

[1806 rows x 3 columns]


### QUESTION 11

In [39]:
def find_ids_within_ten_percentage_threshold(df, reference_id):
    avg_ref = df[df['id_start'] == reference_id]['distance'].mean()
    avg_dist = df.groupby('id_start')['distance'].mean()
    bound_lower = avg_ref * 0.9
    bound_upper = avg_ref * 1.1
    id_in_threshold = avg_dist[(avg_dist >= bound_lower) & 
                                         (avg_dist <= bound_upper)].index.tolist()
    if reference_id in id_in_threshold:
        id_in_threshold.remove(reference_id)
    id_in_threshold.sort()
    return id_in_threshold

In [40]:
file = 'dataset-2.csv'
distance_matrix = calculate_distance_matrix(file)
unroll_df = unroll_distance_matrix(distance_matrix)

reference_id = 1004356  # Example reference ID
result = find_ids_within_ten_percentage_threshold(unroll_df, reference_id)
print(f"IDs within 10% threshold of reference ID {reference_id}: {result}")

IDs within 10% threshold of reference ID 1004356: [1001428, 1001430, 1001432, 1001434, 1001436, 1001437, 1001438, 1001440, 1001442, 1001444, 1001446, 1001448, 1001488, 1004354, 1004355]


### QUESTION 12

In [43]:
def calculate_toll_rate(df):
    coefficient_for_rate={'moto':0.8, 'car':1.2,'rv':1.5,'bus':2.2,'truck':3.6}
    for type_of_veh , rate in coefficient_for_rate.items():
        df[type_of_veh]=df['distance'] * rate 
    round_to_column = ['moto','car','rv','bus','truck']
    df[round_to_column] = df[round_to_column].round(2)
    return df

In [44]:
result_df = calculate_toll_rate(unroll_df)
print(result_df)

      id_start   id_end  distance    moto     car      rv     bus   truck
0      1001400  1001402       9.7    7.76   11.64   14.55   21.34   34.92
1      1001400  1001404      29.9   23.92   35.88   44.85   65.78  107.64
2      1001400  1001406      45.9   36.72   55.08   68.85  100.98  165.24
3      1001400  1001408      67.6   54.08   81.12  101.40  148.72  243.36
4      1001400  1001410      78.7   62.96   94.44  118.05  173.14  283.32
...        ...      ...       ...     ...     ...     ...     ...     ...
1801   1004356  1001470     159.8  127.84  191.76  239.70  351.56  575.28
1802   1004356  1001472     175.8  140.64  210.96  263.70  386.76  632.88
1803   1004356  1001488       4.0    3.20    4.80    6.00    8.80   14.40
1804   1004356  1004354       2.0    1.60    2.40    3.00    4.40    7.20
1805   1004356  1004355       4.0    3.20    4.80    6.00    8.80   14.40

[1806 rows x 8 columns]


### QUESTION 13

In [62]:
import pandas as pd
from datetime import time

def calculate_base_toll_rates(df):
    if not isinstance(df, pd.DataFrame):
        raise TypeError("Input must be a pandas DataFrame")
    df['distance'] = pd.to_numeric(df['distance'], errors='coerce')
    df['moto'] = df['distance'] * 0.8
    df['car'] = df['distance'] * 1.2
    df['rv'] = df['distance'] * 1.5
    df['bus'] = df['distance'] * 2.2
    df['truck'] = df['distance'] * 3.6

    return df

In [63]:
def calculate_time_based_toll_rates(df):
    time_intervals = [
        (time(0, 0, 0), time(10, 0, 0), 0.8),
        (time(10, 0, 0), time(18, 0, 0), 1.2),
        (time(18, 0, 0), time(23, 59, 59), 0.8)
    ]
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    rows_new = []

    for _, row in df.iterrows():
        for day in days:
            if day in ['Saturday', 'Sunday']:
                new_row = row.copy()
                new_row['start_day'] = day
                new_row['end_day'] = day
                new_row['start_time'] = time(0, 0, 0)
                new_row['end_time'] = time(23, 59, 59)
                for vehicle in ['moto', 'car', 'rv', 'bus', 'truck']:
                    new_row[vehicle] *= 0.7
                rows_new.append(new_row)
            else:
                for start_time, end_time, discount in time_intervals:
                    new_row = row.copy()
                    new_row['start_day'] = day
                    new_row['end_day'] = day
                    new_row['start_time'] = start_time
                    new_row['end_time'] = end_time
                    for vehicle in ['moto', 'car', 'rv', 'bus', 'truck']:
                        new_row[vehicle] *= discount
                    rows_new.append(new_row)
    result_df = pd.DataFrame(rows_new)
    vehicle_columns = ['moto', 'car', 'rv', 'bus', 'truck']
    result_df[vehicle_columns] = result_df[vehicle_columns].round(2)

    return result_df

In [64]:
# Calculate base and time-based toll rates
df = calculate_base_toll_rates(df)
time_based_df = calculate_time_based_toll_rates(df)

# Print and save the resulting DataFrame
print(time_based_df)
time_based_df.to_csv('time_based_toll_rates.csv', index=False)


   distance   moto    car     rv    bus  truck  start_day    end_day  \
0     100.0   64.0   96.0  120.0  176.0  288.0     Monday     Monday   
0     100.0   96.0  144.0  180.0  264.0  432.0     Monday     Monday   
0     100.0   64.0   96.0  120.0  176.0  288.0     Monday     Monday   
0     100.0   64.0   96.0  120.0  176.0  288.0    Tuesday    Tuesday   
0     100.0   96.0  144.0  180.0  264.0  432.0    Tuesday    Tuesday   
0     100.0   64.0   96.0  120.0  176.0  288.0    Tuesday    Tuesday   
0     100.0   64.0   96.0  120.0  176.0  288.0  Wednesday  Wednesday   
0     100.0   96.0  144.0  180.0  264.0  432.0  Wednesday  Wednesday   
0     100.0   64.0   96.0  120.0  176.0  288.0  Wednesday  Wednesday   
0     100.0   64.0   96.0  120.0  176.0  288.0   Thursday   Thursday   
0     100.0   96.0  144.0  180.0  264.0  432.0   Thursday   Thursday   
0     100.0   64.0   96.0  120.0  176.0  288.0   Thursday   Thursday   
0     100.0   64.0   96.0  120.0  176.0  288.0     Friday     Fr